In [1]:
import { ChatMessageHistory } from "langchain/stores/message/in_memory";
import { HumanMessage, AIMessage } from "@langchain/core/messages";

const history = new ChatMessageHistory();

await history.addMessage(new HumanMessage("hi"));
await history.addMessage(new AIMessage("What can I do for you?"));

const messages = await history.getMessages();

console.log(messages);


[
  HumanMessage {
    lc_serializable: true,
    lc_kwargs: { content: "hi", additional_kwargs: {}, response_metadata: {} },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "hi",
    name: undefined,
    additional_kwargs: {},
    response_metadata: {}
  },
  AIMessage {
    lc_serializable: true,
    lc_kwargs: {
      content: "What can I do for you?",
      additional_kwargs: {},
      response_metadata: {}
    },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "What can I do for you?",
    name: undefined,
    additional_kwargs: {},
    response_metadata: {}
  }
]


In [2]:
// 自动维护 chat history

import { RunnableWithMessageHistory } from "@langchain/core/runnables";
import { ChatPromptTemplate, MessagesPlaceholder } from "@langchain/core/prompts";
import { ChatOpenAI } from "@langchain/openai";
import { ChatMessageHistory } from "langchain/stores/message/in_memory";

const chatModel = new ChatOpenAI({
  configuration: {
    baseURL: 'https://api.shellgpt.top/v1',
  },
});

const prompt = ChatPromptTemplate.fromMessages([
    ["system", "You are a helpful assistant. Answer all questions to the best of your ability."],
    new MessagesPlaceholder("history_message"),
    ["human","{input}"]
]);

const history = new ChatMessageHistory();
const chain = prompt.pipe(chatModel)

const chainWithHistory = new RunnableWithMessageHistory({
  runnable: chain,
  getMessageHistory: (_sessionId) => history,
  inputMessagesKey: "input",
  historyMessagesKey: "history_message",
});

In [3]:
const res1 = await chainWithHistory.invoke({
  input: "hi, my name is Kai"
},{
  configurable: { sessionId: "none" }
})

In [4]:
res1

AIMessage {
  lc_serializable: true,
  lc_kwargs: {
    content: "Hello Kai! How can I assist you today?",
    tool_calls: [],
    invalid_tool_calls: [],
    additional_kwargs: { function_call: undefined, tool_calls: undefined },
    response_metadata: {}
  },
  lc_namespace: [ "langchain_core", "messages" ],
  content: "Hello Kai! How can I assist you today?",
  name: undefined,
  additional_kwargs: { function_call: undefined, tool_calls: undefined },
  response_metadata: {
    tokenUsage: { completionTokens: 10, promptTokens: 33, totalTokens: 43 },
    finish_reason: "stop"
  },
  tool_calls: [],
  invalid_tool_calls: []
}

In [5]:
const res2 = await chainWithHistory.invoke({
  input: "我的名字叫什么？"
},{
  configurable: { sessionId: "none" }
})

In [6]:
res2

AIMessage {
  lc_serializable: true,
  lc_kwargs: {
    content: "您刚刚告诉我您的名字是Kai。所以，您的名字确实是Kai。有任何其他问题我能帮到您吗？",
    tool_calls: [],
    invalid_tool_calls: [],
    additional_kwargs: { function_call: undefined, tool_calls: undefined },
    response_metadata: {}
  },
  lc_namespace: [ "langchain_core", "messages" ],
  content: "您刚刚告诉我您的名字是Kai。所以，您的名字确实是Kai。有任何其他问题我能帮到您吗？",
  name: undefined,
  additional_kwargs: { function_call: undefined, tool_calls: undefined },
  response_metadata: {
    tokenUsage: { completionTokens: 44, promptTokens: 60, totalTokens: 104 },
    finish_reason: "stop"
  },
  tool_calls: [],
  invalid_tool_calls: []
}

In [7]:
await history.getMessages()

[
  HumanMessage {
    lc_serializable: true,
    lc_kwargs: {
      content: "hi, my name is Kai",
      additional_kwargs: {},
      response_metadata: {}
    },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "hi, my name is Kai",
    name: undefined,
    additional_kwargs: {},
    response_metadata: {}
  },
  AIMessage {
    lc_serializable: true,
    lc_kwargs: {
      content: "Hello Kai! How can I assist you today?",
      tool_calls: [],
      invalid_tool_calls: [],
      additional_kwargs: { function_call: undefined, tool_calls: undefined },
      response_metadata: {}
    },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "Hello Kai! How can I assist you today?",
    name: undefined,
    additional_kwargs: { function_call: undefined, tool_calls: undefined },
    response_metadata: {
      tokenUsage: { completionTokens: 10, promptTokens: 33, totalTokens: 43 },
      finish_reason: "stop"
    },
    tool_calls: [],
    invalid_tool_calls: []


: 

In [1]:
import { JSONChatHistory } from "./JSONChatHistory/index.ts"
import { AIMessage, HumanMessage } from "@langchain/core/messages";

const history = new JSONChatHistory({
    dir: "chat_data",
    sessionId: "test"
})


await history.addMessages([
  new HumanMessage("Hi, 我叫小明"),
  new AIMessage("你好"),
]);

const messages = await history.getMessages();
console.log(messages)


[
  HumanMessage {
    lc_serializable: true,
    lc_kwargs: {
      content: "Hi, 我叫小明",
      additional_kwargs: {},
      response_metadata: {}
    },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "Hi, 我叫小明",
    name: undefined,
    additional_kwargs: {},
    response_metadata: {}
  },
  AIMessage {
    lc_serializable: true,
    lc_kwargs: { content: "你好", additional_kwargs: {}, response_metadata: {} },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "你好",
    name: undefined,
    additional_kwargs: {},
    response_metadata: {}
  }
]


In [2]:
import { ChatOpenAI } from "@langchain/openai";
import { BufferMemory } from "langchain/memory";
import { ConversationChain } from "langchain/chains";


const chatModel = new ChatOpenAI({
  configuration: {
    baseURL: 'https://api.shellgpt.top/v1',
  },
  temperature: 0.2,
});

const memory = new BufferMemory({
    chatHistory: history
});
const chain = new ConversationChain({ llm: chatModel, memory: memory });
const res1 = await chain.call({ input: "我叫什么？" });
console.log('res1 :', res1);


res1 : { response: "你叫小明。" }
